In [ ]:
!pip install flaml --progress-bar off

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
ss = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

X = train_df.drop(['target', 'id'], axis = 1).values
y = train_df['target'].values
X_test = test_df.drop('id', axis = 1).values

del train_df, test_df

In [ ]:
from flaml import AutoML

automl = AutoML()

automl_settings = {
    "time_budget" : 7.5*3600, # in seconds
    "metric" : 'roc_auc',
    "task" : 'classification',
    "estimator_list": ['lgbm'],
    "log_file_name" : "flaml.log"
}

automl.fit(X, y, **automl_settings)

train_preds = automl.predict_proba(X)[:, -1]
pd.DataFrame({
    'id': [i for i in range(X.shape[0])],
    'target': train_preds
}).to_csv('train_pred.csv', index=False)

test_preds = automl.predict_proba(X_test)[:, -1]

ss['target'] = test_preds
ss.to_csv('test_pred.csv', index=False)
ss.head()